In [ ]:
# pip install pyautogen
# !pip install python-dotenv

In [9]:
# Import required libraries

import autogen
from dotenv import load_dotenv
import os

load_dotenv()
config_list = [{"model": "gpt-4", "api_key": os.getenv("OPENAI_API_KEY")}]
llm_config = {'config_list': config_list}

In [10]:
# Create the tasks

financial_tasks = [
    'What are the current stock prices of NVIDIA and TESLA? How is the performance over the past month in terms of percentage change?',
    'Investigate possible reasons of the stock performance'
]

writing_tasks = [
    'Write an engaging blog post using the information provided to you.'
]

In [11]:
# Create the agents

financial_assistant = autogen.AssistantAgent(
    name = 'Financial_Assistant',
    llm_config = llm_config
)

research_assistant= autogen.AssistantAgent(
    name = 'Researcher',
    llm_config = llm_config
)

writer = autogen.AssistantAgent(
    name = 'writer',
    llm_config = llm_config,
    system_message = '''
        You are a professional writer, known for you insightful
        and engaging articles.
        You transform complex concepts into compelling narratives.
        Print 'TERMINATE' at the end when evertything is done.
    '''
)


user = autogen.UserProxyAgent(
    name = 'User',
    human_input_mode = 'NEVER',
    is_termination_msg = lambda x: x.get('content','') and x.get('content','').rstrip().endswith('TERMINATE'),
    code_execution_config = {
        'last_n_messages': 1,
        'work_dir': 'tasks',
        'use_docker': False
    }
)

In [12]:
# Initiate the chat

chat_results = user.initiate_chats(
    [
        {
            'recipient': financial_assistant,
            'message': financial_tasks[0], # pass 1st task to financial assistant
            'clear_history': True,
            'silent': False,
            'summary_method': 'last_msg',
        },
        {
            'recipient': research_assistant,
            'message': financial_tasks[1], # pass 2nd task to research assistant
            'summary_method': 'reflection_with_llm',
        },
        {
            'recipient': writer,
            'message': writing_tasks[0],
            'carryover': 'I want to include a figure or a table of data in the blog post.',
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
User (to Financial_Assistant):

What are the current stock prices of NVIDIA and TESLA? How is the performance over the past month in terms of percentage change?

--------------------------------------------------------------------------------
Financial_Assistant (to User):

To access the real-time stock data, we will use the `yfinance` module in Python. This module allows you to download stock information from Yahoo Finance.
The script first downloads the historical data for the past month for NVIDIA and TESLA. Then it calculates the percentage change in the last month.

Please execute the following Python script:

```python
import yfinance as yf
import datetime

# Get today's date
end = datetime.date.today()

# Calculate the date a month before
start = end - datetime.timedelta(days=30)

# Download th